# Simulation

In [ ]:
import  fenics as fe
from dolfin import *
from mshr import *


R = 0.5  # Radius of the cylinder
L = 2   #length of the cylinder

D_a = 1
A = 0.1
u_B = 1

In [ ]:
constant_shift = fe.Constant(0.01)

#Données de précision
Number_of_elemnts = 60 #définit la précision du calcul et augmente le temps de calcul
Degree_of_elements = 1 #définit la précision du calcul

# Define the geometry and mesh
cylinder = Cylinder(fe.Point(0, 0, 0), fe.Point(L, 0, 0), R, R)

mesh = generate_mesh(cylinder, Number_of_elemnts) # Generates a mesh with Number_of_elements divisions
lagrange_vector_space_first_order = fe.FunctionSpace(
        mesh,
        "Lagrange",
        Degree_of_elements,
    )
print("mesh generated")

n_trial = fe.TrialFunction(lagrange_vector_space_first_order) #guess function
v_test = fe.TestFunction(lagrange_vector_space_first_order) #test function v

#definition of the weak form of the problem
weak_form_rhs = - u_B * constant_shift * v_test * fe.ds  
weak_form_lhs = (
    fe.dot(fe.grad(n_trial), fe.grad(v_test)) * fe.dx #consequence of the IPP
    -
    u_B * n_trial * v_test * fe.ds #consequence of the boundary conditions
    -
    A * n_trial * v_test * fe.dx 
)

In [ ]:
#solving
n_solution = fe.Function(lagrange_vector_space_first_order)
fe.solve(
    weak_form_lhs == weak_form_rhs,
    n_solution,
)
print(fe.assemble(n_solution * fe.dx))

# Results analysis

In [ ]:
print("center_density = ",  n_solution(Point(0.5, 0, 0)))
plot(n_solution)
plt.show()

In [ ]:
# Circle density
# Vertical cut through x=0 plane (y-z plane)
ny, nz = 100, 100
y = np.linspace(-R, R, ny)   # adjust to your cylinder size
z = np.linspace(-R, R, nz)
Y, Z = np.meshgrid(y, z)
X = np.zeros_like(Y) + L/2

values = np.zeros_like(Y)
for i in range(ny):
    for j in range(nz):
        try:
            values[j, i] = n_solution(Point(X[j,i], Y[j,i], Z[j,i]))
        except RuntimeError:   # point outside domain
            values[j, i] = np.nan

plt.figure()
plt.imshow(values, extent=[y.min(), y.max(), z.min(), z.max()],
           origin="lower", aspect="auto")
plt.colorbar(label="n_solution")
plt.xlabel("Radius 1")
plt.ylabel("Radius 2")
plt.title("Horizontal cut (x=L/2 plane)")
plt.show()


In [ ]:
# Vertical cut through z=0 plane
ny, nz = 100, 100
y = np.linspace(-R, R, ny)   # adjust to your cylinder size
z = np.linspace(0, L, nz)
Y, Z = np.meshgrid(y, z)
X = np.zeros_like(Y)

values = np.zeros_like(Y)
for j in range(nz):
    for i in range(ny):
        try:
            values[j, i] = n_solution(Point(Z[j,i], Y[j,i], X[j,i]))
        except RuntimeError:   # point outside domain
            values[j, i] = np.nan

plt.figure()
plt.imshow(values, extent=[y.min(), y.max(), z.min(), z.max()],
           origin="lower", aspect="auto")
plt.colorbar(label="n_solution")
plt.xlabel("Length")
plt.ylabel("Radius")
plt.title("Vertical cut (z=0 plane)")
plt.show()


In [ ]:
ny, nz = 100, 100
y =0   # adjust to your cylinder size
z = np.linspace(0, L, nz)


values = []
for i in range(nz):
        try:
            values.append(n_solution(Point(z[i], 0, 0)))
        except RuntimeError:   # point outside domain
            values.append(np.nan)

plt.figure()
plt.plot(z, values)
plt.xlabel("length")
plt.ylabel("density")
plt.title("Vertical cut (x=0 plane)")
plt.show()